In [1]:
!nvidia-smi

Sat Feb 10 03:49:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A4500               On  | 00000000:82:00.0 Off |                  Off |
| 30%   26C    P8              18W / 200W |      2MiB / 20470MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
#!pip install vllm

In [2]:
#!pip install pandas

In [3]:
#!pip install tqdm

In [4]:
from vllm import LLM, SamplingParams
import torch
import pandas as pd
import time
import os
import tqdm
#from prompts import questions
#from rich.progress import track

tensor_parallel_size = int(os.environ.get("DEVICES", "1"))

sampling_params = SamplingParams(max_tokens=200)

llm = LLM(
    "mistralai/Mistral-7B-v0.1",
    tensor_parallel_size=tensor_parallel_size,
    dtype=torch.bfloat16,
    gpu_memory_utilization=1.0,
    max_model_len=14736,
    download_dir="/workspace/"
)

/tmp/ipykernel_689/4175378454.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


INFO 02-10 03:57:56 llm_engine.py:72] Initializing an LLM engine with config: model='mistralai/Mistral-7B-v0.1', tokenizer='mistralai/Mistral-7B-v0.1', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=14736, download_dir='/workspace/', load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, seed=0)
INFO 02-10 03:58:00 weight_utils.py:164] Using model weights format ['*.safetensors']
INFO 02-10 03:58:23 llm_engine.py:322] # GPU blocks: 2051, # CPU blocks: 2048
INFO 02-10 03:58:25 model_runner.py:632] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-10 03:58:25 model_runner.py:636] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decr

In [5]:
out = llm.generate("This is me warming up the model", sampling_params=sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.63s/it]


In [6]:
#out[0].prompt,
out[0].outputs[0].text

', something you should do before any portrait shoot, for many reasons.  #1 the lighting you may have set up may be cold, i.e. it is un lit and to warm it up take a hit or two with a flash or hot light. #2, models need to loosen up a little, they can be pretty stiff before the camera, coming out of the bottom of the hourly rate they are in (if not working through an agent), and any cold air a photographer can snap out of them is, at least in their minds, time taken away from their rate. Natural lighting can help to break the ice a little too.'

In [7]:
import tqdm

questions = ["This is me warming up the model"]*100


print(f"running inference through {len(questions)} prompt.")

results = []
for q in tqdm.tqdm(questions):
    t0 = time.perf_counter()
    output = llm.generate(q, sampling_params=sampling_params)[0]
    t1 = time.perf_counter()
    results.append(
        {"time": t1 - t0, "tokens_generated": len(output.outputs[0].token_ids)}
    )

df = pd.DataFrame(results)
df["tokens_per_sec"] = df.tokens_generated / df.time
print(f"Average tokens/sec: {df.tokens_per_sec.mean(): .3f}")
df.to_csv(f"vllm-benchmark-{tensor_parallel_size}GPUs.csv", index=False)

running inference through 100 prompt.


100%|██████████| 100/100 [08:36<00:00,  5.17s/it]

Average tokens/sec:  36.672


In [8]:
df

,time,tokens_generated,tokens_per_sec
0,4.362613,160,36.675270
1,5.432370,200,36.816341
2,5.437150,200,36.783977
3,5.437862,200,36.779163
4,5.435945,200,36.792131
...,...,...,...
95,5.457376,200,36.647646
96,5.458475,200,36.640271
97,3.981285,146,36.671580
98,5.458321,200,36.641302
